In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [ ]:
TRAIN_DATA_PATH = './data/train_images'
TEST_DATA_PATH = './data/test_images'

In [ ]:
# Image properties

img_width = 300
img_height = 300

In [ ]:
# Hyperparameters

batch_size = 32

In [13]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [14]:
df_train['id_code'] = './data/train_images/' + df_train['id_code'] + '.png'
df_test['id_code'] = './data/test_images/' + df_test['id_code'] + 'png'

In [15]:
df_train.head()

,id_code,diagnosis
0,./data/train_images/000c1434d8d7.png,2
1,./data/train_images/001639a390f0.png,4
2,./data/train_images/0024cdab0c1e.png,1
3,./data/train_images/002c21358ce6.png,0
4,./data/train_images/005b95c28852.png,0


In [16]:
df_test.head()

,id_code
0,./data/test_images/0005cfc8afb6png
1,./data/test_images/003f0afdcd15png
2,./data/test_images/006efc72b638png
3,./data/test_images/00836aaacf06png
4,./data/test_images/009245722fa4png


In [ ]:
# TODO: Show sample image

In [ ]:
# Data generator

data_generator = ImageDataGenerator(rotation_range=360,
                                   horizontal_flip = True,
                                   vertival_flip = True,
                                   valiation_split=0.15)

train_generator = data_generator.flow_from_dataframe(df_train,
                                                    x_col='id_code',
                                                    y_col='diagnosis',
                                                    directory = TRAIN_DATA_PATH,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='other')

